In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Problem 1**

In [2]:
!pip install PyMuPDF
!pip install konlpy
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import io
import fitz
import re
from konlpy.tag import Mecab, Komoran
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.8/493.8 kB 30.0 MB/s eta 0:00:00


In [ ]:
'''
problem 1 : 101-120 페이지에 있는 모든 리포트들에 대해 제목과 본문을 Komoran으로
분석한 뒤, 형태소의 종류별 개수를 구하라(종목명, 날짜, 제목, 본문, 명사개수 등)한 df에 저장
'''
# 100-120페이지 주소 가져오기       -(1)

# 제목 + 본문 df에 저장하기        -(2)

def pages_parsing(page_url, inner_content_links):
  page_html = urlopen(page_url)
  bs_obj = BeautifulSoup(page_html, 'html.parser')
  page_reports = bs_obj.find_all("td", {"style":"padding-left:10"})
  page_pdf = bs_obj.find_all("td",{"class":"file"})
  page_date = bs_obj.find_all("td",{"class":"date"})

  # 리포트 내부 값 저장
  page_report_title = []
  page_report_date = []

  for i in range(30):
    page_report_title.append(page_reports[i].find("a")['title'])
    page_report_date.append(page_date[2*i].text)

  page_df = pd.DataFrame(
    {  '종목명': page_report_title,
      '날짜' :page_report_date,

    }
    )

  page_report_title = []
  page_report_content = []

  for link in inner_content_links:
    # 각 링크마다 접속
    page_link = urlopen(link)
    parser = BeautifulSoup(page_link, 'html.parser')

    # 본문 크롤링해서 가져오기
    try:
      title = parser.find("td", {'class':'view_cnt'}).find_all('p')[0].get_text(strip=True)
      content = parser.find("td", {'class':'view_cnt'}).find_all('p')[1].get_text(strip=True)

    except:
      title = None
      content = None
    page_report_title.append(title)
    page_report_content.append(content)
    # print(content)

  page_df['제목'] = page_report_title
  page_df['본문'] = page_report_content

  return page_df


def inner_content(url):
  # url : 메인 페이지(1,2,3 ...)
  # 위에서부터 본문 주소를 다 불러오기
  page_html = urlopen(url)
  bs_obj = BeautifulSoup(page_html, 'html.parser')

  page_reports = bs_obj.find_all("td")
  # print(page_reports)

  content_links =  ['https://finance.naver.com/research/'+a['href'] for a in bs_obj.find_all('a') if a['href'].startswith('company_read')]

  print(content_links)
  return content_links


# __main__

from collections import Counter

page101_url = 'https://finance.naver.com/research/company_list.naver'

page_list = [] # 101-120p url
for i in range(1,21):
  page_list.append(page101_url+'?&page='+str(i+100))

result_6_df = pd.DataFrame()

for page in page_list:
  print(i)
  page_content_link = inner_content(page)
  page_df = pages_parsing(page, page_content_link)
  result_6_df = pd.concat([result_6_df, page_df])
  print(result_6_df.shape)
  i+=1



In [5]:




# ==========================================================
komoran = Komoran()
pos_counts6 = []


for content in result_6_df['본문']:
  pos = komoran.pos(str(content))
  pos_counter = Counter(tag for _, tag in pos)
  pos_counts6.append(pos_counter)

result_6_pos = pd.DataFrame(pos_counts6).fillna(0)
# Specify lsuffix and rsuffix to differentiate overlapping columns
result_6_df = result_6_df.join(result_6_pos, lsuffix='_caller', rsuffix='_other')
result_6_df.head()
# print(result_6_df)
result_6_df.to_csv("output.csv")

In [14]:
lab5_df = result_6_df # 이름 변경해서 아래에서 사용
lab5_df.head()
lab5_df.to_csv("output.csv")

오늘 가격, 30일 뒤 가격을 데이터프레임에 추가

In [17]:
!pip install pykrx
from pykrx import stock
import pandas as pd

# 날짜 기준 코스피 상장 종목 티커랑 이름 가져오기
tickers = stock.get_market_ticker_list(market = 'KOSPI')

# 티커 : 종목명 딕셔너리로 저장
data = {"Ticker":tickers, "Name":[stock.get_market_ticker_name(ticker) for ticker in tickers]}

# df변환
df = pd.DataFrame(data)
df


,Ticker,Name
0,095570,AJ네트웍스
1,006840,AK홀딩스
2,027410,BGF
3,282330,BGF리테일
4,138930,BNK금융지주
...,...,...
954,079980,휴비스
955,005010,휴스틸
956,000540,흥국화재
957,000545,흥국화재우


In [18]:
def get_close_price(stock_name:str, target_date: str, df : pd.DataFrame):
  # 종목 명 티커 찾기
  ticker = df.loc[df['Name']==stock_name, 'Ticker'] # ticker df 에서 가져온거
  if ticker.empty:
    return f"{stock_name}에 해당하는 종목이 없습니다"

  ticker = ticker.values[0]

  #주어진 날짜로부터 가장 가까운 영업일 종가 가져오기
  close_price = stock.get_market_ohlcv_by_date(target_date, target_date, ticker)

  if close_price.empty:
    next_trading_day = stock.get_nearest_business_day_in_a_week(data=target_date, prev=False)
    close_price = stock.get_market_ohlcv_by_date(next_trading_day, next_trading_day, ticker)
    result_date = next_trading_day

  else:
    result_date = target_date

  # return f"{stock_name} ({ticker})의 {result_date} 종가 : {close_price['종가'].values[0]}원"
  return close_price['종가'].values[0]


In [19]:
import pandas as pd

# 예시로 제공된 get_close_price 함수
def get_close_price(stock_name:str, target_date: str, df : pd.DataFrame):
    # 종목명에 해당하는 티커 찾기
    ticker = df.loc[df['Name'] == stock_name, 'Ticker']
    if ticker.empty:
        return f"{stock_name}에 해당하는 종목이 없습니다"

    ticker = ticker.values[0]

    # 주어진 날짜로부터 가장 가까운 영업일 종가 가져오기
    close_price = stock.get_market_ohlcv_by_date(target_date, target_date, ticker)

    if close_price.empty:
        # target_date를 'YYYYMMDD' 형식으로 변환
        target_date_formatted = target_date.replace('-', '')
        next_trading_day = stock.get_nearest_business_day_in_a_week(target_date_formatted, prev=False) # 수정된 부분
        close_price = stock.get_market_ohlcv_by_date(next_trading_day, next_trading_day, ticker)
        result_date = next_trading_day
    else:
        result_date = target_date

    return close_price['종가'].values[0]

# lab5_df에 종목명과 날짜가 있다면 해당 종목들의 종가를 반환하는 함수
def get_prices_for_lab5(lab5_df: pd.DataFrame, target_date: str, df: pd.DataFrame):

    prices = []
    target_date_30 = pd.to_datetime(target_date) + pd.DateOffset(days=30)

    for stock_name in lab5_df['종목명']:
      price = get_close_price(stock_name, target_date, df)
      price_30 = get_close_price(stock_name, target_date_30, df)
      prices.append({
      'stock_name': stock_name,
      'price': price,
      'price + 30days': price_30
}
      )

    # 결과를 데이터프레임으로 변환
    return pd.DataFrame(prices)

target_date = '2024-10-19'
prices_df = get_prices_for_lab5(lab5_df, target_date, df)
print(prices_df)

    stock_name                   price          price + 30days
0         대우건설                    3795                    3600
1          넷마블                   59500                   47900
2         유한양행                  160000                  119700
3         롯데칠성                  120700                  115900
4       현대오토에버                  138700                  141100
..         ...                     ...                     ...
595    마이크로디지탈  마이크로디지탈에 해당하는 종목이 없습니다  마이크로디지탈에 해당하는 종목이 없습니다
596       에코아이     에코아이에 해당하는 종목이 없습니다     에코아이에 해당하는 종목이 없습니다
597   신세계인터내셔날                   13140                   10670
598        오리온                   99500                   97600
599     SK네트웍스                    5110                    4345

[600 rows x 3 columns]


In [20]:
lab5 = pd.merge(lab5_df,prices_df, left_on='종목명', right_on='stock_name', how='inner' , suffixes=('_lab5', '_price'))
print(lab5.drop_duplicates())

lab5 = lab5.drop_duplicates()

           종목명        날짜                                 제목  \
0         대우건설  24.07.31   2Q24 영업이익, 컨센서스(1,237억원) -15% 하회   
6          넷마블  24.07.31        변화로 인한 재평가로 인한 큰 폭 주가 상승 전망   
8         유한양행  24.07.31          투자의견 매수, 목표주가 134,000원 유지   
10        롯데칠성  24.07.31         투자의견 BUY, 목표주가 170,000원 유지   
13      현대오토에버  24.07.31  투자의견 매수 유지, 목표주가 230,000원 상향(+5%)   
...        ...       ...                                ...   
3066   마이크로디지탈  24.07.18                               None   
3067      에코아이  24.07.18                               None   
3068  신세계인터내셔날  24.07.18        2Q24 Preview: 화장품 선방, 의류 부진   
3070       오리온  24.07.17          6월 Re: 매출 공백 이슈는 어느정도 마무리   
3071    SK네트웍스  24.07.17       목표주가 7,000원으로 하향, 투자의견 매수 유지   

                                                     본문  SN_caller  SL_caller  \
0     2Q24 매출액 2.8조원(YoY -14%), 영업이익 1,048억원(YoY -52...       20.0       10.0   
6     오랜 기간 부진을 끝내고 완전히 달라진 실적과 신작, 그리고 상장사 중 가장 안정적...        0.0        0.0   


In [22]:
lab5.to_csv("output.csv")
from google.colab import files
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
lab5

,종목명,날짜,제목,본문,SN_caller,SL_caller,NNP_caller,NNG_caller,SF_caller,SS_caller,...,EF_other,VX_other,VCN_other,SH_other,IC_other,JKV_other,stock_name,price,price + 30days,price_change
0,대우건설,24.07.31,"2Q24 영업이익, 컨센서스(1,237억원) -15% 하회","2Q24 매출액 2.8조원(YoY -14%), 영업이익 1,048억원(YoY -52...",20.0,10.0,19.0,36.0,8.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,대우건설,3795,3600,False
6,넷마블,24.07.31,변화로 인한 재평가로 인한 큰 폭 주가 상승 전망,"오랜 기간 부진을 끝내고 완전히 달라진 실적과 신작, 그리고 상장사 중 가장 안정적...",0.0,0.0,4.0,9.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,넷마블,59500,47900,False
8,유한양행,24.07.31,"투자의견 매수, 목표주가 134,000원 유지",목표주가는 SOTP 방식으로 산출. 영업가치는 25년 추정 EBITDA에 Targe...,9.0,6.0,11.0,14.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,유한양행,160000,119700,False
10,롯데칠성,24.07.31,"투자의견 BUY, 목표주가 170,000원 유지",상반기 부진했던 별도법인은 1) 3분기 계절적 성수기 진입 2) 6월 음료 주요 제...,9.0,3.0,13.0,46.0,1.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,롯데칠성,120700,115900,False
13,현대오토에버,24.07.31,"투자의견 매수 유지, 목표주가 230,000원 상향(+5%)","TP 23만원은 2024~25년 평균 EPS 7,268원에 Target PER 32...",24.0,14.0,22.0,47.0,3.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,현대오토에버,138700,141100,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066,마이크로디지탈,24.07.18,None,None,3.0,2.0,13.0,14.0,2.0,2.0,...,2.0,0.0,0.0,0.0,0.0,0.0,마이크로디지탈,마이크로디지탈에 해당하는 종목이 없습니다,마이크로디지탈에 해당하는 종목이 없습니다,False
3067,에코아이,24.07.18,None,None,2.0,0.0,3.0,33.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,에코아이,에코아이에 해당하는 종목이 없습니다,에코아이에 해당하는 종목이 없습니다,False
3068,신세계인터내셔날,24.07.18,"2Q24 Preview: 화장품 선방, 의류 부진","신세계인터내셔날의 2Q24 실적은 매출액 3,203억원(YoY -4%), 영업이익 ...",0.0,2.0,13.0,29.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,신세계인터내셔날,13140,10670,False
3070,오리온,24.07.17,6월 Re: 매출 공백 이슈는 어느정도 마무리,"6월 법인별 단순 합산 매출액 및 영업이익은 각각 2,288억원(YoY 0.5%),...",33.0,16.0,16.0,35.0,14.0,21.0,...,4.0,0.0,0.0,0.0,0.0,0.0,오리온,99500,97600,False


#**problem 2**

In [40]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import train_test_split

# 데이터 로드 (가정: 데이터프레임 df)
# df는 종목명, 날짜, 오늘 가격, 30일 뒤 가격, 제목, 본문, 명사개수, 형용사개수 등의 컬럼을 포함한 데이터프레임

# 목표: 오늘 가격 대비 30일 뒤 가격이 상승했는지 여부
# 30일 뒤 가격 상승 여부를 나타내는 컬럼을 생성
lab5['price_change'] = (lab5['price + 30days'] > lab5['price']).astype(int)
# lab5.head()

import pandas as pd


# 형태소 종류의 컬럼 목록
morpheme_columns = [
    'SN_caller', 'SL_caller', 'NNP_caller', 'NNG_caller', 'SF_caller', 'SS_caller',
    'SW_caller', 'SP_caller', 'NR_caller', 'NNB_caller', 'JKB_caller', 'JX_caller',
    'XSN_caller', 'JC_caller', 'JKO_caller', 'MAG_caller', 'JKG_caller', 'VV_caller',
    'EC_caller', 'XSV_caller', 'EP_caller', 'MAJ_caller', 'ETM_caller', 'XPN_caller',
    'MM_caller', 'JKS_caller', 'VCP_caller', 'VA_caller', 'NA_caller', 'XR_caller',
    'ETN_caller', 'XSA_caller', 'SO_caller', 'NP_caller', 'EF_caller', 'VX_caller',
    'VCN_caller', 'SH_caller', 'IC_caller', 'JKV_caller', 'SN_other', 'SL_other',
    'NNP_other', 'NNG_other', 'SF_other', 'SS_other', 'SW_other', 'SP_other', 'NR_other',
    'NNB_other', 'JKB_other', 'JX_other', 'XSN_other', 'JC_other', 'JKO_other', 'MAG_other',
    'JKG_other', 'VV_other', 'EC_other', 'XSV_other', 'EP_other', 'MAJ_other', 'ETM_other',
    'XPN_other', 'MM_other', 'JKS_other', 'VCP_other', 'VA_other', 'NA_other', 'XR_other',
    'ETN_other', 'XSA_other', 'SO_other', 'NP_other', 'EF_other', 'VX_other', 'VCN_other',
    'SH_other', 'IC_other', 'JKV_other'
]

# 새 데이터프레임을 저장할 리스트
new_data = []

# 각 행에 대해 형태소 종류별 비율 계산
for _, row in lab5.iterrows():
    # 각 행의 전체 형태소 개수 (caller와 other를 모두 합산)
    total_caller = row[morpheme_columns].sum()

    # caller 비율과 other 비율 계산
    row_ratios = {}
    for col in morpheme_columns:
        row_ratios[col] = row[col] / total_caller if total_caller > 0 else 0  # 0으로 나누는 경우 방지

    # 비율만 저장한 새로운 행 추가
    new_data.append(row_ratios)

# 비율을 저장할 새 데이터프레임
df_ratios = pd.DataFrame(new_data, columns=morpheme_columns)

# df_ratios는 각 형태소 종류별 비율이 계산된 데이터프레임

df_ratios = df_ratios.reset_index(drop=True) # index drop안할시 오류로 nan값 생성됨
lab5 = lab5.reset_index(drop=True)
df_ratios['price_change'] = lab5['price_change']

print(df_ratios)
# print(lab5['price_change'])


     SN_caller  SL_caller  NNP_caller  NNG_caller  SF_caller  SS_caller  \
0     0.060606   0.030303    0.057576    0.109091   0.024242   0.042424   
1     0.000000   0.000000    0.060606    0.136364   0.000000   0.000000   
2     0.057692   0.038462    0.070513    0.089744   0.044872   0.000000   
3     0.039130   0.013043    0.056522    0.200000   0.004348   0.034783   
4     0.060606   0.035354    0.055556    0.118687   0.007576   0.037879   
..         ...        ...         ...         ...        ...        ...   
595   0.024590   0.016393    0.106557    0.114754   0.016393   0.016393   
596   0.010870   0.000000    0.016304    0.179348   0.010870   0.010870   
597   0.000000   0.013699    0.089041    0.198630   0.013699   0.000000   
598   0.076389   0.037037    0.037037    0.081019   0.032407   0.048611   
599   0.000000   0.500000    0.000000    0.000000   0.000000   0.000000   

     SW_caller  SP_caller  NR_caller  NNB_caller  ...  XSA_other  SO_other  \
0     0.030303   0.02

In [47]:
# 독립 변수 준비: 형태소 비율과 관련된 변수들
X = lab5[morpheme_columns]

# 종속 변수 준비: 가격 상승 여부
y = lab5['price_change']

# 모델 학습
rf = RandomForestClassifier(n_estimators=50, min_samples_split=2, max_depth=None, bootstrap = True, random_state=42, class_weight='balanced')
rf.fit(X, y)

# 예측 및 평가
y_pred = rf.predict(X)

# Confusion Matrix 계산
cm = confusion_matrix(y, y_pred)

# F1-Score 계산
f1 = f1_score(y, y_pred)

# Feature Importance 출력
feature_importances = rf.feature_importances_

# 중요 변수 출력
important_features = sorted(zip(X.columns, feature_importances), key=lambda x: x[1], reverse=True)[:3]

# 결과 출력
print("Confusion Matrix:")
print(cm)
print("\nF1-Score:", f1)
print("\nTop 3 Important Features:")
for feature, importance in important_features:
    print(f"{feature}: {importance}")



Confusion Matrix:
[[313 175]
 [ 47  65]]

F1-Score: 0.3693181818181818

Top 3 Important Features:
SL_other: 0.09396534786725404
SL_caller: 0.05517387690492319
SN_other: 0.03885053806102999


# **problem 3**

In [48]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score


# 30일 뒤 가격이 상승했는지 여부 (1: 상승, 0: 하락)
lab5['상승여부'] = np.where(lab5['price + 30days'] > lab5['price'], 1, 0)

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer()  # 불용어 제거

# 제목과 본문을 합쳐서 TF-IDF 벡터화
lab5['제목'] = lab5['제목'].fillna('')
lab5['본문'] = lab5['본문'].fillna('')
tfidf_matrix = tfidf_vectorizer.fit_transform(lab5['제목'] + ' ' + lab5['본문'])

# TF-IDF 매트릭스는 희소 행렬(sparse matrix)로 반환되므로 이를 DataFrame으로 변환
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# 목표 변수 (타겟 변수)
y = lab5['상승여부']

# 로지스틱 회귀 모델 학습
logreg_model = LogisticRegression(C=100,max_iter=10000, random_state=42)
logreg_model.fit(tfidf_df, y)




# grid_search를 사용한 모델 파라미터 조정(조정 안 했을시 f1-score : 0.0)
'''
# param_grid = {'C': [0.01, 0.1, 1, 10, 100]}

# # 그리드 서치 실행
# from sklearn.model_selection import GridSearchCV
# grid_search = GridSearchCV(logreg_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# # 학습 데이터로 그리드 서치 실행
# grid_search.fit(tfidf_df, y)

# # 최적의 C 값 출력
# print("최적의 C 값:", grid_search.best_params_['C'])

# # 최적 모델로 예측 및 성능 평가
# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(tfidf_df)

# # 평가 결과 출력 (예: accuracy)
# from sklearn.metrics import accuracy_score
# print("Accuracy:", accuracy_score(y, y_pred))
'''
# c=100일시 최대 정확도가 나와 c=100으로 파라미터 수정 후 모델 학습 진행


# coef_를 사용하여 계수 (특징 중요도)
coef = logreg_model.coef_.flatten()  # (1, n_features)에서 (n_features,)로 평탄화

# 특성 이름
feature_names = tfidf_vectorizer.get_feature_names_out()

# 계수가 양인 긍정적인 키워드 상위 5개
positive_coefficients = [(feature_names[i], coef[i]) for i in range(len(coef)) if coef[i] > 0]
positive_coefficients = sorted(positive_coefficients, key=lambda x: x[1], reverse=True)[:5]

# 계수가 음인 부정적인 키워드 상위 5개
negative_coefficients = [(feature_names[i], coef[i]) for i in range(len(coef)) if coef[i] < 0]
negative_coefficients = sorted(negative_coefficients, key=lambda x: x[1])[:5]

# 예측
y_pred = logreg_model.predict(tfidf_df)

# 성능 평가 (Confusion Matrix, F1-Score)
cm = confusion_matrix(y, y_pred)
f1 = f1_score(y, y_pred)

# 결과 출력
print("긍정 키워드 상위 5개:")
for word, coef_val in positive_coefficients:
    print(f"{word}: {coef_val}")

print("\n부정 키워드 상위 5개:")
for word, coef_val in negative_coefficients:
    print(f"{word}: {coef_val}")

print("\nConfusion Matrix:")
print(cm)

print("\nF1-Score:", f1)


긍정 키워드 상위 5개:
북미: 5.97628756448718
전동화: 5.698553309202797
21: 5.195662451322861
프리뷰: 4.967107526584318
감소: 4.7252149074476

부정 키워드 상위 5개:
전망: -5.797102279758341
qoq: -3.215134375837839
실적이: -3.14047003928314
연결: -3.095408641864154
sk: -2.93202834079469

Confusion Matrix:
[[488   0]
 [  2 110]]

F1-Score: 0.990990990990991


# **problem 4**

In [49]:
!pip install gensim scikit-learn

In [50]:
# w2v

import gensim
from gensim.models import Word2Vec
import numpy as np

# 제목과 본문을 공백 기준으로 토큰화한 리스트 (예: lab5['제목']과 lab5['본문']의 텍스트 데이터)
texts = lab5['제목'].apply(lambda x: x.split()) + lab5['본문'].apply(lambda x: x.split())

# Word2Vec 모델 학습
word2vec_model = Word2Vec(sentences=texts, vector_size=100, window=5, min_count=1, workers=4, sg=0)

# 각 문서(제목 + 본문)에 대한 벡터를 평균하여 문서 벡터 생성
def get_word2vec_vector(text, model):
    words = text.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)  # 벡터가 없으면 0으로 채움
    return np.mean(word_vectors, axis=0)

lab5['word2vec_vector'] = lab5['제목'] + ' ' + lab5['본문']
lab5['word2vec_vector'] = lab5['word2vec_vector'].apply(lambda x: get_word2vec_vector(x, word2vec_model))
X_word2vec = np.vstack(lab5['word2vec_vector'].values)  # Word2Vec 매트릭스


In [51]:
# ft

from gensim.models import FastText

# FastText 모델 학습
fasttext_model = FastText(sentences=texts, vector_size=100, window=3, min_count=1,workers=4, sg=0) #grid search cv로 튜닝한 파라미터

# 각 문서(제목 + 본문)에 대한 벡터를 평균하여 문서 벡터 생성
def get_fasttext_vector(text, model):
    words = text.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)  # 벡터가 없으면 0으로 채움
    return np.mean(word_vectors, axis=0)

lab5['fasttext_vector'] = lab5['제목'] + ' ' + lab5['본문']
lab5['fasttext_vector'] = lab5['fasttext_vector'].apply(lambda x: get_fasttext_vector(x, fasttext_model))
X_fasttext = np.vstack(lab5['fasttext_vector'].values)  # FastText 매트릭스


In [52]:
#모델 학습

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.model_selection import train_test_split

# 타겟 변수 설정 (예: 30일 뒤 가격 상승 여부)
y = lab5['상승여부']  # '상승여부'는 상승 여부를 나타내는 열로 가정

# 학습 데이터와 테스트 데이터로 분리 (교차 검증 없이 모든 데이터를 학습 데이터로 사용한다고 하셨으므로 전체 데이터 사용)
X_train = X_word2vec  # Word2Vec 매트릭스
X_train_fasttext = X_fasttext  # FastText 매트릭스

# Word2Vec 모델 학습
rf_word2vec = RandomForestClassifier(n_estimators=100, random_state=42)
rf_word2vec.fit(X_train, y)

# FastText 모델 학습
rf_fasttext = RandomForestClassifier(n_estimators=100, random_state=42)
rf_fasttext.fit(X_train_fasttext, y)

# 예측
y_pred_word2vec = rf_word2vec.predict(X_train)
y_pred_fasttext = rf_fasttext.predict(X_train)

# 평가 (Confusion Matrix와 F1-Score)
print("Word2Vec 모델 성능:")
print(confusion_matrix(y, y_pred_word2vec))
print("F1-Score:", f1_score(y, y_pred_word2vec))

print("\nFastText 모델 성능:")
print(confusion_matrix(y, y_pred_fasttext))
print("F1-Score:", f1_score(y, y_pred_fasttext))


Word2Vec 모델 성능:
[[488   0]
 [  2 110]]
F1-Score: 0.990990990990991

FastText 모델 성능:
[[150 338]
 [ 31  81]]
F1-Score: 0.3050847457627119
